In [3]:
import os
import pandas as pd
from binance.client import Client

In [4]:
API_KEY = os.getenv('BINANCE_API')
API_SECRET = os.getenv('BINANCE_SECRET')

In [5]:
client = Client(api_key = API_KEY, api_secret = API_SECRET, tld = "us")

In [6]:
client.ping()

{}

In [7]:
client.get_server_time()

{'serverTime': 1671847410020}

In [8]:
account = client.get_account()
account["accountType"]  #Account Type

'SPOT'

In [9]:
pd.to_datetime(account["updateTime"], unit = "ms") #Last Update -> UTC Time

Timestamp('2022-09-27 16:12:32.888000')

In [10]:
account["balances"] #List Account Balances

[{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '0.00000246', 'locked': '0.00000000'},
 {'asset': 'USD', 'free': '0.0731', 'locked': '0.0000'},
 {'asset': 'XRP', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '0.12797628', 'locked': '0.00000000'},
 {'asset': 'BCH', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'ADA', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'XLM', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BAT', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'ETC', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'ZRX', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'BNB', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'LINK', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'REP', 'free': '0.00000000', 'locked': '0.00000000'},
 {'asset': 'RVN', 'free': '0.00

In [11]:
df = pd.DataFrame(account["balances"]) #Convert balances from JSON into a dataframe
df

,asset,free,locked
0,BTC,0.00000000,0.00000000
1,ETH,0.00000246,0.00000000
2,USD,0.0731,0.0000
3,XRP,0.00000000,0.00000000
4,USDT,0.12797628,0.00000000
...,...,...,...
153,APT,0.00000000,0.00000000
154,MASK,0.00000000,0.00000000
155,BOSON,0.00000000,0.00000000
156,MXC,0.00000000,0.00000000


In [12]:
df.info()  #Sanity check on the data types of each column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   asset   158 non-null    object
 1   free    158 non-null    object
 2   locked  158 non-null    object
dtypes: object(3)
memory usage: 3.8+ KB


In [13]:
df.free = pd.to_numeric(df.free, errors="coerce") # Convert the free and locked columns into numbers since they should be numeric
df.locked = pd.to_numeric(df.locked, errors="coerce")

In [14]:
df.loc[df.free > 0]  #show list of columns matching criteria https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.loc.html

,asset,free,locked
1,ETH,0.000002,0.0
2,USD,0.073100,0.0
4,USDT,0.127976,0.0
44,HNT,1632.763740,0.0


In [15]:
client.get_asset_balance(asset = "HNT")  #get a specific asset balance from the binance API client

{'asset': 'HNT', 'free': '1632.76374000', 'locked': '0.00000000'}

In [16]:
float(client.get_asset_balance(asset = "HNT")["free"]) #get the free amount of an asset and convert to float

1632.76374

In [17]:
snap = client.get_account_snapshot(type = "SPOT") #Binance has daily account snapshots with account main points. Retrieve that for viewing
snap

{'code': 200, 'msg': '', 'snapshotVos': []}

In [18]:
snap = pd.json_normalize(snap["snapshotVos"]) #convert the snapshot into a normalized dataframe
snap

""


In [20]:
client.get_symbol_info(symbol = "BUSDUSDT")

{'symbol': 'BUSDUSDT',
 'status': 'TRADING',
 'baseAsset': 'BUSD',
 'baseAssetPrecision': 8,
 'quoteAsset': 'USDT',
 'quotePrecision': 8,
 'quoteAssetPrecision': 8,
 'baseCommissionPrecision': 8,
 'quoteCommissionPrecision': 8,
 'orderTypes': ['LIMIT',
  'LIMIT_MAKER',
  'MARKET',
  'STOP_LOSS_LIMIT',
  'TAKE_PROFIT_LIMIT'],
 'icebergAllowed': True,
 'ocoAllowed': True,
 'quoteOrderQtyMarketAllowed': True,
 'allowTrailingStop': True,
 'cancelReplaceAllowed': True,
 'isSpotTradingAllowed': True,
 'isMarginTradingAllowed': False,
 'filters': [{'filterType': 'PRICE_FILTER',
   'minPrice': '0.00010000',
   'maxPrice': '1000.00000000',
   'tickSize': '0.00010000'},
  {'filterType': 'PERCENT_PRICE',
   'multiplierUp': '5',
   'multiplierDown': '0.2',
   'avgPriceMins': 5},
  {'filterType': 'LOT_SIZE',
   'minQty': '0.01000000',
   'maxQty': '9000000.00000000',
   'stepSize': '0.01000000'},
  {'filterType': 'MIN_NOTIONAL',
   'minNotional': '10.00000000',
   'applyToMarket': True,
   'avgPric

In [24]:
client.get_symbol_ticker(symbol = "HNTUSDT") # get current ticker price

{'symbol': 'HNTUSDT', 'price': '1.87858000'}

In [28]:
prices = client.get_all_tickers() #get all tickers and create a dataframe for them
df = pd.DataFrame(prices)
df

,symbol,price
0,BTCUSD,16819.0300
1,ETHUSD,1216.7100
2,XRPUSD,0.2970
3,BCHUSD,102.1900
4,LTCUSD,65.6700
...,...,...
336,ADAUSDC,0.25660000
337,MXCUSDT,0.02755000
338,MXCUSD,0.0358
339,JAMUSDT,0.00212000


In [30]:
df[df.symbol.str.contains("BTC") & df.symbol.str.contains("USD")]

,symbol,price
0,BTCUSD,16819.0300
6,BTCUSDT,16813.56000000
35,BTCBUSD,16823.90000000
107,BTCUSDC,16818.00000000


In [31]:
last24 = client.get_ticker(symbol = "BTCUSDT") #24 Price change statistic
last24

{'symbol': 'BTCUSDT',
 'priceChange': '11.29000000',
 'priceChangePercent': '0.067',
 'weightedAvgPrice': '16837.96301381',
 'prevClosePrice': '16807.41000000',
 'lastPrice': '16821.24000000',
 'lastQty': '0.00579000',
 'bidPrice': '16821.34000000',
 'bidQty': '0.00594300',
 'askPrice': '16824.49000000',
 'askQty': '0.00594300',
 'openPrice': '16809.95000000',
 'highPrice': '16945.58000000',
 'lowPrice': '16761.57000000',
 'volume': '934.94689200',
 'quoteVolume': '15742601.18737248',
 'openTime': 1671763699326,
 'closeTime': 1671850099326,
 'firstId': 17521778,
 'lastId': 17541913,
 'count': 20136}

In [34]:
pd.to_datetime(last24["openTime"], unit = "ms") # Show datetime in a readable format

Timestamp('2022-12-23 02:48:19.326000')

In [39]:
(float(last24["lastPrice"])/float(last24["openPrice"]) -1) *100

0.06716260310113409